In [1]:
import os
import numpy as np
import pandas as pd
from scipy.signal import butter, filtfilt, iirnotch

# Configuration
BASE_PATH = '/kaggle/input/mtcaic3-phase-ii'
OUTPUT_DIR = './preprocessed'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Filter design
def design_filters(fs=250.0):
    # Bandpass 1-40 Hz
    bp_b, bp_a = butter(4, [1/(fs/2), 40/(fs/2)], btype='band')
    # Notch at 50 Hz
    notch_b, notch_a = iirnotch(50/(fs/2), Q=30)
    return bp_b, bp_a, notch_b, notch_a

# Preprocessing steps for one trial
def preprocess_trial(df):
    eeg_cols = ['FZ','C3','CZ','C4','PZ','PO7','OZ','PO8']
    motion_cols = ['AccX','AccY','AccZ','Gyro1','Gyro2','Gyro3']
    val_col = 'Validation'

    # 1) Motion artifact detection
    motion_mag = np.sqrt((df[motion_cols]**2).sum(axis=1))
    mot_thresh = np.percentile(motion_mag, 95)
    bad_mask = motion_mag > mot_thresh

    # 2) Mask EEG
    data = df[eeg_cols].copy().values
    data[bad_mask, :] = np.nan
    data[df[val_col] == 0, :] = np.nan

    # 3) Interpolation
    for ch in range(data.shape[1]):
        col = data[:, ch]
        nans = np.isnan(col)
        if nans.all():
            continue
        idx = np.arange(len(col))
        data[nans, ch] = np.interp(idx[nans], idx[~nans], col[~nans])

    # 4) Filtering
    bp_b, bp_a, notch_b, notch_a = design_filters()
    for ch in range(data.shape[1]):
        data[:, ch] = filtfilt(bp_b, bp_a, data[:, ch])
        data[:, ch] = filtfilt(notch_b, notch_a, data[:, ch])

    # 5) Baseline correction (first 0.5s)
    bs = int(0.5 * 250)
    baseline = data[:bs].mean(axis=0)
    data -= baseline
    return data

# Load index DataFrame
def load_index(fname, label_col=True):
    df = pd.read_csv(os.path.join(BASE_PATH, fname))
    cols = ['id','subject_id','task','trial_session','trial'] + (['label'] if label_col else [])
    return df[cols]

# Process a split, grouping by task
def process_split(df, has_label=True):
    data_dict = {'MI': {'X': [], 'y': []}, 'SSVEP': {'X': [], 'y': []}}
    for _, row in df.iterrows():
        # Identify folder
        idx = row['id']
        split = 'train' if idx <= 4800 else 'validation' if idx <= 4900 else 'test'
        # Load EEGdata
        path = os.path.join(BASE_PATH, row['task'], split, row['subject_id'], str(row['trial_session']), 'EEGdata.csv')
        df_eeg = pd.read_csv(path)
        # Extract correct segment
        n_samp = 2250 if row['task']=='MI' else 1750
        start = (row['trial']-1)*n_samp
        seg = df_eeg.iloc[int(start):int(start+n_samp)].reset_index(drop=True)
        proc = preprocess_trial(seg)
        data_dict[row['task']]['X'].append(proc.T)
        if has_label:
            data_dict[row['task']]['y'].append(row['label'])
    # Stack
    for t in data_dict:
        X = np.stack(data_dict[t]['X'])
        y = np.array(data_dict[t]['y']) if has_label else None
        data_dict[t]['X'] = X
        data_dict[t]['y'] = y
    return data_dict

# Execute processing and save
for fname, label_col in [('train.csv', True), ('validation.csv', True), ('test.csv', False)]:
    df_idx = load_index(fname, label_col)
    results = process_split(df_idx, label_col)
    for task, d in results.items():
        out_file = f"{os.path.splitext(fname)[0]}_{task}.npz"
        path = os.path.join(OUTPUT_DIR, out_file)
        if label_col:
            np.savez_compressed(path, X=d['X'], y=d['y'])
        else:
            np.savez_compressed(path, X=d['X'])
        print(f"Saved {out_file}: X={d['X'].shape}" + (f", y={d['y'].shape}" if d['y'] is not None else ''))
print('Preprocessing complete.')

Saved train_MI.npz: X=(2400, 8, 2250), y=(2400,)
Saved train_SSVEP.npz: X=(2400, 8, 1750), y=(2400,)
Saved validation_MI.npz: X=(50, 8, 2250), y=(50,)
Saved validation_SSVEP.npz: X=(50, 8, 1750), y=(50,)
Saved test_MI.npz: X=(100, 8, 2250)
Saved test_SSVEP.npz: X=(100, 8, 1750)
Preprocessing complete.


In [3]:
import os
import numpy as np
import tensorflow as tf

from sklearn.metrics import f1_score, classification_report
from mne.decoding import CSP
from tensorflow import keras
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    CSVLogger,
    LearningRateScheduler,
)

# --- 0) (Optional) Reproducibility ---
# os.environ['PYTHONHASHSEED'] = '0'
# np.random.seed(0)
# tf.random.set_seed(0)
# os.environ['TF_DETERMINISTIC_OPS'] = '1'
# os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

# --- 1) Config ---
data_dir   = './preprocessed'
output_dir = './models'
os.makedirs(output_dir, exist_ok=True)

# --- 2) Load & binarize labels ---
train_npz = np.load(os.path.join(data_dir, 'train_MI.npz'))
val_npz   = np.load(os.path.join(data_dir, 'validation_MI.npz'))
X_train, y_train = train_npz['X'], train_npz['y']
X_val,   y_val   = val_npz['X'],   val_npz['y']

y_train_bin = (y_train == 'Right').astype(int)
y_val_bin   = (y_val   == 'Right').astype(int)

# --- 3) CSP (4 components) ---
csp = CSP(n_components=4, log=False, norm_trace=False)
csp.fit(X_train, y_train_bin)
W = csp.filters_[:4]
def apply_csp(X): return np.stack([W.dot(ep) for ep in X], axis=0)

Xtr = apply_csp(X_train).transpose(0,2,1).astype('float32')  # (n, T, 4)
Xvl = apply_csp(X_val)  .transpose(0,2,1).astype('float32')

# --- 4) For Model3 only: add channel axis → (n, T, F=4, 1) ---
Xtr_spec = Xtr[..., np.newaxis]
Xvl_spec = Xvl[..., np.newaxis]

# --- 5) One‑hot labels ---
ytr_oh = keras.utils.to_categorical(y_train_bin, 2)
yvl_oh = keras.utils.to_categorical(y_val_bin,   2)

# --- 6) Data‑augmentation gens ---
def aug_gen(X, y, seed=0, batch_size=32):
    n   = X.shape[0]
    rng = np.random.RandomState(seed)
    while True:
        idx = rng.randint(0, n, batch_size)
        bx, by = X[idx].copy(), y[idx]
        bx += rng.normal(0, 0.005, bx.shape)
        yield bx, by

train_gen_1d = aug_gen(Xtr,   ytr_oh, seed=0, batch_size=64)
train_gen_2d = aug_gen(Xtr_spec, ytr_oh, seed=1, batch_size=64)

steps_1d = len(Xtr)      // 64
steps_2d = len(Xtr_spec) // 64

# --- 7) LR schedule ---
def cosine_lr(epoch, lr_max=5e-5, epochs=200):
    return lr_max * (1 + np.cos(np.pi * epoch / epochs)) / 2

# --- 8) F1Score metric ---
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name="f1_score", **kwargs):
        super().__init__(name=name, **kwargs)
        # create the state variables up front
        self.tp = self.add_weight(name="tp", initializer="zeros")
        self.fp = self.add_weight(name="fp", initializer="zeros")
        self.fn = self.add_weight(name="fn", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        preds  = tf.argmax(y_pred, axis=1)
        labels = tf.argmax(y_true, axis=1)
        preds  = tf.cast(preds, tf.int32)
        labels = tf.cast(labels, tf.int32)

        tp = tf.reduce_sum(
            tf.cast(tf.logical_and(preds == 1, labels == 1), tf.float32)
        )
        fp = tf.reduce_sum(
            tf.cast(tf.logical_and(preds == 1, labels == 0), tf.float32)
        )
        fn = tf.reduce_sum(
            tf.cast(tf.logical_and(preds == 0, labels == 1), tf.float32)
        )

        self.tp.assign_add(tp)
        self.fp.assign_add(fp)
        self.fn.assign_add(fn)

    def result(self):
        precision = self.tp / (self.tp + self.fp + tf.keras.backend.epsilon())
        recall    = self.tp / (self.tp + self.fn + tf.keras.backend.epsilon())
        return 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())

    def reset_states(self):
        self.tp.assign(0.0)
        self.fp.assign(0.0)
        self.fn.assign(0.0)

# --- 9) Callbacks factory ---
def get_callbacks(name):
    return [
        EarlyStopping("val_f1_score", mode="max", patience=20, restore_best_weights=True),
        ModelCheckpoint(
            os.path.join(output_dir, f"best_{name}.h5"),
            "val_f1_score", mode="max", save_best_only=True
        ),
        CSVLogger(os.path.join(output_dir, f"log_{name}.csv")),
        LearningRateScheduler(cosine_lr)
    ]

# --- 10) Model builders (all output 2 classes) ---

def build_modelA(input_shape):
    m = keras.Sequential([
        layers.Input(input_shape),
        layers.Conv1D(32, 5, activation="relu", padding="same"),
        layers.BatchNormalization(), layers.MaxPool1D(2),
        layers.Conv1D(64, 5, activation="relu", padding="same"),
        layers.BatchNormalization(), layers.MaxPool1D(2),
        layers.Conv1D(128,5,activation="relu",padding="same"),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling1D(),
        layers.Dense(64, activation="relu", 
                     kernel_regularizer=regularizers.l2(1e-4)),
        layers.Dropout(0.7),
        layers.Dense(2, activation="softmax"),
    ])
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_modelB(input_shape):
    inp = layers.Input(input_shape)
    x = inp
    for f in [16,32,64,128,256]:
        x = layers.Conv1D(f,3,activation="relu",padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPool1D(2)(x)
    x = layers.Flatten()(x)
    for u in [128,64,32]:
        x = layers.Dense(u, activation="relu",
                         kernel_regularizer=regularizers.l2(1e-4))(x)
        x = layers.Dropout(0.5)(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp, out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model1(input_shape):
    inp = layers.Input(input_shape+(1,))
    x = layers.Concatenate()([inp, inp, inp])
    x = layers.Resizing(32,32)(x)
    base = keras.applications.ResNet50(
        include_top=False, weights="imagenet",
        input_shape=(32,32,3), pooling="avg"
    )
    base.trainable = False
    x = base(x)
    x = layers.Reshape((1, x.shape[-1]))(x)
    for _ in range(7):
        x = layers.Conv1D(64,3,activation="relu",padding="same")(x)
    x = layers.MultiHeadAttention(num_heads=4,key_dim=32)(x,x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64,activation="relu")(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model2(input_shape):
    inp = layers.Input(input_shape)
    x = inp
    for _ in range(3):
        x = layers.Conv1D(32,3,activation="elu",padding="same")(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64,activation="elu")(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model3(input_shape):
    inp = layers.Input(input_shape)  # (T, F, 1)
    x = inp
    for _ in range(5):
        x = layers.Conv2D(32,(3,3),activation="relu",padding="same")(x)
    x = layers.Flatten()(x)
    for u in [128,64,32]:
        x = layers.Dense(u, activation="relu")(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model4(input_shape):
    inp = layers.Input(input_shape)
    x = inp
    for _ in range(3):
        x = layers.Conv1D(64,3,activation="relu",padding="same")(x)
    x = layers.LSTM(128)(x)
    for _ in range(4):
        x = layers.Dense(64, activation="relu")(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model5(input_shape):
    inp = layers.Input(input_shape)
    x = inp
    for _ in range(7):
        x = layers.Conv1D(64,3,activation="elu",padding="same")(x)
    x = layers.Flatten()(x)
    for _ in range(3):
        x = layers.Dense(64, activation="elu")(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model6(input_shape):
    C3, C4 = 0,2
    eeg_in = layers.Input(input_shape)
    c3 = layers.Lambda(lambda x: x[:,:,C3:C3+1])(eeg_in)
    c4 = layers.Lambda(lambda x: x[:,:,C4:C4+1])(eeg_in)
    def branch():
        return models.Sequential([
            layers.Conv1D(16,250,activation="relu",padding="same"),
            layers.MaxPool1D(3),
            layers.Conv1D(32,50,activation="relu",padding="same"),
            layers.GlobalAveragePooling1D()
        ])
    b3, b4 = branch()(c3), branch()(c4)
    x = layers.Concatenate()([b3,b4])
    for _ in range(4):
        x = layers.Dense(64,activation="relu")(x)
    out = layers.Dense(2,activation="softmax")(x)
    m = models.Model(eeg_in,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

# --- 11) Train & eval loop ---
builders = {
    'oldA': build_modelA,
    'oldB': build_modelB,
    'model1': build_model1,
    'model2': build_model2,
    'model3': build_model3,
    'model4': build_model4,
    'model5': build_model5,
    'model6': build_model6,
}

results = {}
shape_1d = Xtr.shape[1:]      # (T, 4)
shape_2d = Xtr_spec.shape[1:] # (T, 4, 1)

for name, build_fn in builders.items():
    print(f"\n>>> Training {name}")
    if name == 'model3':
        model = build_fn(shape_2d)
        gen   = train_gen_2d
        steps = steps_2d
        val_x = Xvl_spec
    else:
        model = build_fn(shape_1d)
        gen   = train_gen_1d
        steps = steps_1d
        val_x = Xvl

    history = model.fit(
        gen,
        steps_per_epoch=steps,
        validation_data=(val_x, yvl_oh),
        epochs=200,
        callbacks=get_callbacks(name),
        verbose=2
    )

    # model now has best weights
    preds = np.argmax(model.predict(val_x), axis=1)
    f1    = f1_score(y_val_bin, preds)
    print(f"{name} →  val F1 = {f1:.4f}")
    print(classification_report(y_val_bin, preds, target_names=['Left','Right']))
    results[name] = (f1, model)

# --- 12) Pick & save final ---
best_name, (best_f1, best_model) = max(results.items(), key=lambda kv: kv[1][0])
print(f"\n=== Final best: {best_name} (F1={best_f1:.4f}) ===")
best_model.save(os.path.join(output_dir, 'best_final.h5'))


Computing rank from data with rank=None
    Using tolerance 7.4e+03 (2.2e-16 eps * 8 dim * 4.2e+18  max singular value)
    Estimated rank (data): 8
    data: rank 8 computed from 8 data channels with 0 projectors
Reducing data rank from 8 -> 8
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.

>>> Training oldA


I0000 00:00:1753238957.756579      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Epoch 1/200


I0000 00:00:1753238963.014795     102 service.cc:148] XLA service 0x7dc4d0036c60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753238963.015438     102 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753238963.457621     102 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753238966.939560     102 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


37/37 - 10s - 280ms/step - accuracy: 0.5072 - f1_score: 0.4937 - loss: 0.9075 - val_accuracy: 0.5200 - val_f1_score: 0.2941 - val_loss: 0.7158 - learning_rate: 0.0010
Epoch 2/200
37/37 - 1s - 28ms/step - accuracy: 0.5013 - f1_score: 0.5065 - loss: 0.7847 - val_accuracy: 0.5000 - val_f1_score: 0.4186 - val_loss: 0.7271 - learning_rate: 9.9994e-04
Epoch 3/200
37/37 - 1s - 27ms/step - accuracy: 0.5093 - f1_score: 0.4939 - loss: 0.7442 - val_accuracy: 0.5600 - val_f1_score: 0.0000e+00 - val_loss: 0.7120 - learning_rate: 9.9969e-04
Epoch 4/200
37/37 - 1s - 27ms/step - accuracy: 0.5152 - f1_score: 0.5291 - loss: 0.7204 - val_accuracy: 0.5400 - val_f1_score: 0.0000e+00 - val_loss: 0.7228 - learning_rate: 9.9914e-04
Epoch 5/200
37/37 - 1s - 27ms/step - accuracy: 0.5215 - f1_score: 0.5509 - loss: 0.7177 - val_accuracy: 0.5000 - val_f1_score: 0.0741 - val_loss: 0.7193 - learning_rate: 9.9815e-04
Epoch 6/200
37/37 - 1s - 27ms/step - accuracy: 0.5296 - f1_score: 0.4828 - loss: 0.7054 - val_accurac

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


37/37 - 5s - 135ms/step - accuracy: 0.4958 - f1_score: 0.4341 - loss: 0.6992 - val_accuracy: 0.5200 - val_f1_score: 0.0769 - val_loss: 0.7043 - learning_rate: 0.0010
Epoch 2/200
37/37 - 1s - 22ms/step - accuracy: 0.5059 - f1_score: 0.5447 - loss: 0.6921 - val_accuracy: 0.4200 - val_f1_score: 0.4528 - val_loss: 0.7090 - learning_rate: 9.9994e-04
Epoch 3/200
37/37 - 1s - 20ms/step - accuracy: 0.5106 - f1_score: 0.5248 - loss: 0.6894 - val_accuracy: 0.4200 - val_f1_score: 0.4528 - val_loss: 0.7454 - learning_rate: 9.9969e-04
Epoch 4/200
37/37 - 1s - 21ms/step - accuracy: 0.5152 - f1_score: 0.6148 - loss: 0.6954 - val_accuracy: 0.3600 - val_f1_score: 0.3043 - val_loss: 0.7109 - learning_rate: 9.9914e-04
Epoch 5/200
37/37 - 1s - 21ms/step - accuracy: 0.5106 - f1_score: 0.4530 - loss: 0.6921 - val_accuracy: 0.5600 - val_f1_score: 0.2667 - val_loss: 0.7138 - learning_rate: 9.9815e-04
Epoch 6/200
37/37 - 1s - 21ms/step - accuracy: 0.5262 - f1_score: 0.3321 - loss: 0.6900 - val_accuracy: 0.5800

In [4]:
import os
import numpy as np
import tensorflow as tf

from sklearn.metrics import f1_score, classification_report
from mne.decoding import CSP
from tensorflow import keras
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger, LearningRateScheduler

# --- 1) Config ---
data_dir = './preprocessed'
output_dir = './models2'
os.makedirs(output_dir, exist_ok=True)

# --- 2) Load data ---
train_npz = np.load(os.path.join(data_dir, 'train_MI.npz'))
val_npz   = np.load(os.path.join(data_dir, 'validation_MI.npz'))
X_train, y_train = train_npz['X'], train_npz['y']
X_val,   y_val   = val_npz['X'],   val_npz['y']

y_train_bin = (y_train == 'Right').astype(int)
y_val_bin   = (y_val   == 'Right').astype(int)

# --- 3) Select MI-related EEG channels: C3, CZ, C4, PZ (indices 2,3,4,5) ---
mi_indices = [1, 2, 3, 4]
X_train_mi = X_train[:, mi_indices, :]
X_val_mi   = X_val[:,   mi_indices, :]

# --- 4) CSP (n_components=4) ---
csp = CSP(n_components=4, log=False, norm_trace=False)
csp.fit(X_train_mi, y_train_bin)
W = csp.filters_[:4]
def apply_csp(X): return np.stack([W.dot(ep) for ep in X], axis=0)

Xtr = apply_csp(X_train_mi).transpose(0, 2, 1).astype('float32')  # (n, T, 4)
Xvl = apply_csp(X_val_mi).transpose(0, 2, 1).astype('float32')
Xtr_spec = Xtr[..., np.newaxis]
Xvl_spec = Xvl[..., np.newaxis]

# --- 5) One-hot encode labels ---
ytr_oh = keras.utils.to_categorical(y_train_bin, 2)
yvl_oh = keras.utils.to_categorical(y_val_bin,   2)

# --- 6) Data augmentation ---
def aug_gen(X, y, seed=0, batch_size=32):
    n   = X.shape[0]
    rng = np.random.RandomState(seed)
    while True:
        idx = rng.randint(0, n, batch_size)
        bx, by = X[idx].copy(), y[idx]
        bx += rng.normal(0, 0.005, bx.shape)
        yield bx, by

train_gen_1d = aug_gen(Xtr, ytr_oh, seed=0, batch_size=64)
train_gen_2d = aug_gen(Xtr_spec, ytr_oh, seed=1, batch_size=64)

steps_1d = len(Xtr)      // 64
steps_2d = len(Xtr_spec) // 64

# --- 7) Learning rate schedule ---
def cosine_lr(epoch, lr_max=5e-5, epochs=200):
    return lr_max * (1 + np.cos(np.pi * epoch / epochs)) / 2

# --- 8) F1 metric ---
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name="f1_score", **kwargs):
        super().__init__(name=name, **kwargs)
        self.tp = self.add_weight(name="tp", initializer="zeros")
        self.fp = self.add_weight(name="fp", initializer="zeros")
        self.fn = self.add_weight(name="fn", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        preds  = tf.argmax(y_pred, axis=1)
        labels = tf.argmax(y_true, axis=1)
        preds  = tf.cast(preds, tf.int32)
        labels = tf.cast(labels, tf.int32)
        tp = tf.reduce_sum(tf.cast(tf.logical_and(preds==1, labels==1), tf.float32))
        fp = tf.reduce_sum(tf.cast(tf.logical_and(preds==1, labels==0), tf.float32))
        fn = tf.reduce_sum(tf.cast(tf.logical_and(preds==0, labels==1), tf.float32))
        self.tp.assign_add(tp)
        self.fp.assign_add(fp)
        self.fn.assign_add(fn)

    def result(self):
        precision = self.tp / (self.tp + self.fp + tf.keras.backend.epsilon())
        recall    = self.tp / (self.tp + self.fn + tf.keras.backend.epsilon())
        return 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())

    def reset_states(self):
        self.tp.assign(0.)
        self.fp.assign(0.)
        self.fn.assign(0.)

# --- 9) Callbacks ---
def get_callbacks(name):
    return [
        EarlyStopping("val_f1_score", mode="max", patience=20, restore_best_weights=True),
        ModelCheckpoint(f"{output_dir}/best_{name}.h5", "val_f1_score", mode="max", save_best_only=True),
        CSVLogger(f"{output_dir}/log_{name}.csv"),
        LearningRateScheduler(cosine_lr)
    ]

# --- 10) Model builders ---

def build_modelA(input_shape):
    m = keras.Sequential([
        layers.Input(input_shape),
        layers.Conv1D(32, 5, activation="relu", padding="same"),
        layers.BatchNormalization(), layers.MaxPool1D(2),
        layers.Conv1D(64, 5, activation="relu", padding="same"),
        layers.BatchNormalization(), layers.MaxPool1D(2),
        layers.Conv1D(128,5,activation="relu",padding="same"),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling1D(),
        layers.Dense(64, activation="relu", 
                     kernel_regularizer=regularizers.l2(1e-4)),
        layers.Dropout(0.7),
        layers.Dense(2, activation="softmax"),
    ])
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_modelB(input_shape):
    inp = layers.Input(input_shape)
    x = inp
    for f in [16,32,64,128,256]:
        x = layers.Conv1D(f,3,activation="relu",padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPool1D(2)(x)
    x = layers.Flatten()(x)
    for u in [128,64,32]:
        x = layers.Dense(u, activation="relu",
                         kernel_regularizer=regularizers.l2(1e-4))(x)
        x = layers.Dropout(0.5)(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp, out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model1(input_shape):
    inp = layers.Input(input_shape+(1,))
    x = layers.Concatenate()([inp, inp, inp])
    x = layers.Resizing(32,32)(x)
    base = keras.applications.ResNet50(
        include_top=False, weights="imagenet",
        input_shape=(32,32,3), pooling="avg"
    )
    base.trainable = False
    x = base(x)
    x = layers.Reshape((1, x.shape[-1]))(x)
    for _ in range(7):
        x = layers.Conv1D(64,3,activation="relu",padding="same")(x)
    x = layers.MultiHeadAttention(num_heads=4,key_dim=32)(x,x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64,activation="relu")(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model2(input_shape):
    inp = layers.Input(input_shape)
    x = inp
    for _ in range(3):
        x = layers.Conv1D(32,3,activation="elu",padding="same")(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64,activation="elu")(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model3(input_shape):
    inp = layers.Input(input_shape)  # (T, F, 1)
    x = inp
    for _ in range(5):
        x = layers.Conv2D(32,(3,3),activation="relu",padding="same")(x)
    x = layers.Flatten()(x)
    for u in [128,64,32]:
        x = layers.Dense(u, activation="relu")(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model4(input_shape):
    inp = layers.Input(input_shape)
    x = inp
    for _ in range(3):
        x = layers.Conv1D(64,3,activation="relu",padding="same")(x)
    x = layers.LSTM(128)(x)
    for _ in range(4):
        x = layers.Dense(64, activation="relu")(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model5(input_shape):
    inp = layers.Input(input_shape)
    x = inp
    for _ in range(7):
        x = layers.Conv1D(64,3,activation="elu",padding="same")(x)
    x = layers.Flatten()(x)
    for _ in range(3):
        x = layers.Dense(64, activation="elu")(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model6(input_shape):
    C3, C4 = 0,2
    eeg_in = layers.Input(input_shape)
    c3 = layers.Lambda(lambda x: x[:,:,C3:C3+1])(eeg_in)
    c4 = layers.Lambda(lambda x: x[:,:,C4:C4+1])(eeg_in)
    def branch():
        return models.Sequential([
            layers.Conv1D(16,250,activation="relu",padding="same"),
            layers.MaxPool1D(3),
            layers.Conv1D(32,50,activation="relu",padding="same"),
            layers.GlobalAveragePooling1D()
        ])
    b3, b4 = branch()(c3), branch()(c4)
    x = layers.Concatenate()([b3,b4])
    for _ in range(4):
        x = layers.Dense(64,activation="relu")(x)
    out = layers.Dense(2,activation="softmax")(x)
    m = models.Model(eeg_in,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m
# --- 11) Train + Evaluate ---
builders = {
    'oldA': build_modelA,
    'oldB': build_modelB,
    'model1': build_model1,
    'model2': build_model2,
    'model3': build_model3,
    'model4': build_model4,
    'model5': build_model5,
    'model6': build_model6,
}

results = {}
shape_1d = Xtr.shape[1:]      # (T, 4)
shape_2d = Xtr_spec.shape[1:] # (T, 4, 1)

for name, build_fn in builders.items():
    print(f"\n>>> Training {name}")
    if name == 'model3':
        model = build_fn(shape_2d)
        gen   = train_gen_2d
        steps = steps_2d
        val_x = Xvl_spec
    else:
        model = build_fn(shape_1d)
        gen   = train_gen_1d
        steps = steps_1d
        val_x = Xvl

    history = model.fit(
        gen, steps_per_epoch=steps,
        validation_data=(val_x, yvl_oh),
        epochs=200,
        callbacks=get_callbacks(name),
        verbose=2
    )

    preds = np.argmax(model.predict(val_x), axis=1)
    f1 = f1_score(y_val_bin, preds)
    print(f"{name} → F1 = {f1:.4f}")
    print(classification_report(y_val_bin, preds, target_names=['Left','Right']))
    results[name] = (f1, model)

# --- 12) Save best ---
best_name, (best_f1, best_model) = max(results.items(), key=lambda kv: kv[1][0])
print(f"\n=== Final best: {best_name} (F1={best_f1:.4f}) ===")
best_model.save(os.path.join(output_dir, 'best_final.h5'))


Computing rank from data with rank=None
    Using tolerance 3.2e+03 (2.2e-16 eps * 4 dim * 3.6e+18  max singular value)
    Estimated rank (data): 4
    data: rank 4 computed from 4 data channels with 0 projectors
Reducing data rank from 4 -> 4
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.

>>> Training oldA
Epoch 1/200
37/37 - 10s - 258ms/step - accuracy: 0.4937 - f1_score: 0.4839 - loss: 0.8629 - val_accuracy: 0.4800 - val_f1_score: 0.2778 - val_loss: 0.7061 - learning_rate: 0.0010
Epoch 2/200
37/37 - 1s - 28ms/step - accuracy: 0.5097 - f1_score: 0.5024 - loss: 0.7739 - val_accuracy: 0.5600 - val_f1_score: 0.3125 - val_loss: 0.6870 - learning_rate: 9.9994e-04
Epoch 3/200
37/37 - 1s - 27ms/step - accuracy: 0.5059 - f1_score: 0.4855 - loss: 0.7470 - val_accuracy: 0.5400 - val_f1_score: 0.2581 - val_loss: 0.6986 - learning_rate: 9.9969e-04
Epoch 4/200
37/37 - 1s - 27ms/step - accuracy: 0.5021 - f1_score: 0.5040 - loss: 0.7307 - v

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


37/37 - 5s - 122ms/step - accuracy: 0.5055 - f1_score: 0.4566 - loss: 0.6949 - val_accuracy: 0.4800 - val_f1_score: 0.4800 - val_loss: 0.7015 - learning_rate: 0.0010
Epoch 2/200
37/37 - 1s - 22ms/step - accuracy: 0.5025 - f1_score: 0.3721 - loss: 0.6944 - val_accuracy: 0.5000 - val_f1_score: 0.3590 - val_loss: 0.6964 - learning_rate: 9.9994e-04
Epoch 3/200
37/37 - 1s - 21ms/step - accuracy: 0.5190 - f1_score: 0.2991 - loss: 0.6915 - val_accuracy: 0.6000 - val_f1_score: 0.4118 - val_loss: 0.6857 - learning_rate: 9.9969e-04
Epoch 4/200
37/37 - 1s - 20ms/step - accuracy: 0.5076 - f1_score: 0.3697 - loss: 0.6942 - val_accuracy: 0.5400 - val_f1_score: 0.3429 - val_loss: 0.6899 - learning_rate: 9.9914e-04
Epoch 5/200
37/37 - 1s - 23ms/step - accuracy: 0.5118 - f1_score: 0.4197 - loss: 0.6927 - val_accuracy: 0.5800 - val_f1_score: 0.5333 - val_loss: 0.7105 - learning_rate: 9.9815e-04
Epoch 6/200
37/37 - 1s - 20ms/step - accuracy: 0.5173 - f1_score: 0.4177 - loss: 0.6936 - val_accuracy: 0.5400

In [6]:
import os
import numpy as np
import tensorflow as tf

from sklearn.metrics import f1_score, classification_report
from mne.decoding import CSP
from tensorflow import keras
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger, LearningRateScheduler

# --- 0) Config ---
data_dir   = './preprocessed'
output_dir = './models3'
os.makedirs(output_dir, exist_ok=True)

eeg_indices = [1, 2, 3, 4]  # C3, CZ, C4, PZ

# --- 1) Load & filter data ---
train_npz = np.load(os.path.join(data_dir, 'train_MI.npz'))
val_npz   = np.load(os.path.join(data_dir, 'validation_MI.npz'))

X_train_all, y_train = train_npz['X'], train_npz['y']
X_val_all,   y_val   = val_npz['X'],   val_npz['y']

# Filter for EEG channels only
X_train_raw = X_train_all[:, eeg_indices, :].transpose(0, 2, 1).astype('float32')  # (n, T, 4)
X_val_raw   = X_val_all[:,   eeg_indices, :].transpose(0, 2, 1).astype('float32')  # (n, T, 4)

# --- 2) Binarize labels ---
y_train_bin = (y_train == 'Right').astype(int)
y_val_bin   = (y_val   == 'Right').astype(int)

# --- 3) CSP for models 3, 4, 5 ---
csp = CSP(n_components=4, log=False, norm_trace=False)
X_train_csp_input = X_train_raw.transpose(0, 2, 1).astype('float64').copy()  # (n, channels, time)
csp.fit(X_train_csp_input, y_train_bin)  # ← NO 'rank' argument

W = csp.filters_[:4]

def apply_csp(X):  # X: (n, T, C)
    return np.stack([W.dot(ep.T) for ep in X], axis=0)


Xtr_csp = apply_csp(X_train_raw).astype('float32')
Xvl_csp = apply_csp(X_val_raw).astype('float32')

Xtr_csp = Xtr_csp.transpose(0, 2, 1)  # (n, T, 4)
Xvl_csp = Xvl_csp.transpose(0, 2, 1)


# For 2D models
Xtr_csp_2d = Xtr_csp[..., np.newaxis]
Xvl_csp_2d = Xvl_csp[..., np.newaxis]

# --- 4) One-hot labels ---
ytr_oh = keras.utils.to_categorical(y_train_bin, 2)
yvl_oh = keras.utils.to_categorical(y_val_bin,   2)

# --- 5) Data augmentation ---
def aug_gen(X, y, seed=0, batch_size=32):
    n = X.shape[0]
    rng = np.random.RandomState(seed)
    while True:
        idx = rng.randint(0, n, batch_size)
        bx, by = X[idx].copy(), y[idx]
        bx += rng.normal(0, 0.005, bx.shape)
        yield bx, by

train_gen_raw   = aug_gen(X_train_raw,  ytr_oh, seed=0, batch_size=64)
train_gen_csp1d = aug_gen(Xtr_csp,      ytr_oh, seed=1, batch_size=64)
train_gen_csp2d = aug_gen(Xtr_csp_2d,   ytr_oh, seed=2, batch_size=64)

steps_raw   = len(X_train_raw)  // 64
steps_csp1d = len(Xtr_csp)      // 64
steps_csp2d = len(Xtr_csp_2d)   // 64

# --- 6) Cosine LR schedule ---
def cosine_lr(epoch, lr_max=5e-5, epochs=200):
    return lr_max * (1 + np.cos(np.pi * epoch / epochs)) / 2

# --- 7) F1 Score Metric ---
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name="f1_score", **kwargs):
        super().__init__(name=name, **kwargs)
        self.tp = self.add_weight(name="tp", initializer="zeros")
        self.fp = self.add_weight(name="fp", initializer="zeros")
        self.fn = self.add_weight(name="fn", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        preds  = tf.argmax(y_pred, axis=1)
        labels = tf.argmax(y_true, axis=1)
        self.tp.assign_add(tf.reduce_sum(tf.cast(tf.logical_and(preds == 1, labels == 1), tf.float32)))
        self.fp.assign_add(tf.reduce_sum(tf.cast(tf.logical_and(preds == 1, labels == 0), tf.float32)))
        self.fn.assign_add(tf.reduce_sum(tf.cast(tf.logical_and(preds == 0, labels == 1), tf.float32)))

    def result(self):
        p = self.tp / (self.tp + self.fp + tf.keras.backend.epsilon())
        r = self.tp / (self.tp + self.fn + tf.keras.backend.epsilon())
        return 2 * (p * r) / (p + r + tf.keras.backend.epsilon())

    def reset_states(self):
        self.tp.assign(0.0)
        self.fp.assign(0.0)
        self.fn.assign(0.0)

# --- 8) Callback factory ---
def get_callbacks(name):
    return [
        EarlyStopping("val_f1_score", mode="max", patience=20, restore_best_weights=True),
        ModelCheckpoint(os.path.join(output_dir, f"best_{name}.h5"),
                        "val_f1_score", mode="max", save_best_only=True),
        CSVLogger(os.path.join(output_dir, f"log_{name}.csv")),
        LearningRateScheduler(cosine_lr)
    ]

# --- 9) Model Builders ---
# [All model builder functions remain unchanged — see previous message if needed.]
def build_modelA(input_shape):
    m = keras.Sequential([
        layers.Input(input_shape),
        layers.Conv1D(32, 5, activation="relu", padding="same"),
        layers.BatchNormalization(), layers.MaxPool1D(2),
        layers.Conv1D(64, 5, activation="relu", padding="same"),
        layers.BatchNormalization(), layers.MaxPool1D(2),
        layers.Conv1D(128,5,activation="relu",padding="same"),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling1D(),
        layers.Dense(64, activation="relu", 
                     kernel_regularizer=regularizers.l2(1e-4)),
        layers.Dropout(0.7),
        layers.Dense(2, activation="softmax"),
    ])
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_modelB(input_shape):
    inp = layers.Input(input_shape)
    x = inp
    for f in [16,32,64,128,256]:
        x = layers.Conv1D(f,3,activation="relu",padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPool1D(2)(x)
    x = layers.Flatten()(x)
    for u in [128,64,32]:
        x = layers.Dense(u, activation="relu",
                         kernel_regularizer=regularizers.l2(1e-4))(x)
        x = layers.Dropout(0.5)(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp, out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model1(input_shape):
    inp = layers.Input(input_shape+(1,))
    x = layers.Concatenate()([inp, inp, inp])
    x = layers.Resizing(32,32)(x)
    base = keras.applications.ResNet50(
        include_top=False, weights="imagenet",
        input_shape=(32,32,3), pooling="avg"
    )
    base.trainable = False
    x = base(x)
    x = layers.Reshape((1, x.shape[-1]))(x)
    for _ in range(7):
        x = layers.Conv1D(64,3,activation="relu",padding="same")(x)
    x = layers.MultiHeadAttention(num_heads=4,key_dim=32)(x,x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64,activation="relu")(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model2(input_shape):
    inp = layers.Input(input_shape)
    x = inp
    for _ in range(3):
        x = layers.Conv1D(32,3,activation="elu",padding="same")(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64,activation="elu")(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model3(input_shape):
    inp = layers.Input(input_shape)  # (T, F, 1)
    x = inp
    for _ in range(5):
        x = layers.Conv2D(32,(3,3),activation="relu",padding="same")(x)
    x = layers.Flatten()(x)
    for u in [128,64,32]:
        x = layers.Dense(u, activation="relu")(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model4(input_shape):
    inp = layers.Input(input_shape)
    x = inp
    for _ in range(3):
        x = layers.Conv1D(64,3,activation="relu",padding="same")(x)
    x = layers.LSTM(128)(x)
    for _ in range(4):
        x = layers.Dense(64, activation="relu")(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model5(input_shape):
    inp = layers.Input(input_shape)
    x = inp
    for _ in range(7):
        x = layers.Conv1D(64,3,activation="elu",padding="same")(x)
    x = layers.Flatten()(x)
    for _ in range(3):
        x = layers.Dense(64, activation="elu")(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model6(input_shape):
    C3, C4 = 0,2
    eeg_in = layers.Input(input_shape)
    c3 = layers.Lambda(lambda x: x[:,:,C3:C3+1])(eeg_in)
    c4 = layers.Lambda(lambda x: x[:,:,C4:C4+1])(eeg_in)
    def branch():
        return models.Sequential([
            layers.Conv1D(16,250,activation="relu",padding="same"),
            layers.MaxPool1D(3),
            layers.Conv1D(32,50,activation="relu",padding="same"),
            layers.GlobalAveragePooling1D()
        ])
    b3, b4 = branch()(c3), branch()(c4)
    x = layers.Concatenate()([b3,b4])
    for _ in range(4):
        x = layers.Dense(64,activation="relu")(x)
    out = layers.Dense(2,activation="softmax")(x)
    m = models.Model(eeg_in,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m
# (You can paste model builders here unchanged if needed)

# --- 10) Train & evaluate ---
builders = {
    'oldA': build_modelA,
    'oldB': build_modelB,
    'model1': build_model1,
    'model2': build_model2,
    'model3': build_model3,
    'model4': build_model4,
    'model5': build_model5,
    'model6': build_model6,
}

results = {}
shape_raw   = X_train_raw.shape[1:]
shape_csp1d = Xtr_csp.shape[1:]
shape_csp2d = Xtr_csp_2d.shape[1:]

for name, build_fn in builders.items():
    print(f"\n>>> Training {name}")
    if name in ['model3']:
        model = build_fn(shape_csp2d)
        gen, steps, val_x = train_gen_csp2d, steps_csp2d, Xvl_csp_2d
    elif name in ['model4', 'model5']:
        model = build_fn(shape_csp1d)
        gen, steps, val_x = train_gen_csp1d, steps_csp1d, Xvl_csp
    else:
        model = build_fn(shape_raw)
        gen, steps, val_x = train_gen_raw, steps_raw, X_val_raw

    model.fit(
        gen, steps_per_epoch=steps,
        validation_data=(val_x, yvl_oh),
        epochs=200, callbacks=get_callbacks(name), verbose=2
    )

    preds = np.argmax(model.predict(val_x), axis=1)
    f1 = f1_score(y_val_bin, preds)
    print(f"{name} → val F1 = {f1:.4f}")
    print(classification_report(y_val_bin, preds, target_names=["Left", "Right"]))
    results[name] = (f1, model)

# --- 11) Save best model ---
best_name, (best_f1, best_model) = max(results.items(), key=lambda kv: kv[1][0])
print(f"\n=== Final best: {best_name} (F1={best_f1:.4f}) ===")
best_model.save(os.path.join(output_dir, 'best_final.h5'))

Computing rank from data with rank=None
    Using tolerance 3.2e+03 (2.2e-16 eps * 4 dim * 3.6e+18  max singular value)
    Estimated rank (data): 4
    data: rank 4 computed from 4 data channels with 0 projectors
Reducing data rank from 4 -> 4
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.

>>> Training oldA
Epoch 1/200
37/37 - 10s - 266ms/step - accuracy: 0.5135 - f1_score: 0.4627 - loss: 0.8540 - val_accuracy: 0.5000 - val_f1_score: 0.4186 - val_loss: 0.8687 - learning_rate: 0.0010
Epoch 2/200
37/37 - 1s - 28ms/step - accuracy: 0.5114 - f1_score: 0.5079 - loss: 0.7700 - val_accuracy: 0.5200 - val_f1_score: 0.5714 - val_loss: 0.7301 - learning_rate: 9.9994e-04
Epoch 3/200
37/37 - 1s - 28ms/step - accuracy: 0.5241 - f1_score: 0.5283 - loss: 0.7350 - val_accuracy: 0.4600 - val_f1_score: 0.6197 - val_loss: 0.8295 - learning_rate: 9.9969e-04
Epoch 4/200
37/37 - 1s - 27ms/step - accuracy: 0.5110 - f1_score: 0.5508 - loss: 0.7238 - v

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


37/37 - 7s - 198ms/step - accuracy: 0.5000 - f1_score: 0.4918 - loss: 1.1224 - val_accuracy: 0.4000 - val_f1_score: 0.5588 - val_loss: 0.7465 - learning_rate: 0.0010
Epoch 2/200
37/37 - 1s - 22ms/step - accuracy: 0.5051 - f1_score: 0.4869 - loss: 0.7420 - val_accuracy: 0.5600 - val_f1_score: 0.0000e+00 - val_loss: 1.0477 - learning_rate: 9.9994e-04
Epoch 3/200
37/37 - 1s - 23ms/step - accuracy: 0.5059 - f1_score: 0.5595 - loss: 0.7280 - val_accuracy: 0.4800 - val_f1_score: 0.3158 - val_loss: 0.7059 - learning_rate: 9.9969e-04
Epoch 4/200
37/37 - 1s - 22ms/step - accuracy: 0.5089 - f1_score: 0.5553 - loss: 0.7356 - val_accuracy: 0.6000 - val_f1_score: 0.3750 - val_loss: 0.6824 - learning_rate: 9.9914e-04
Epoch 5/200
37/37 - 1s - 35ms/step - accuracy: 0.5127 - f1_score: 0.5384 - loss: 0.7244 - val_accuracy: 0.5600 - val_f1_score: 0.6071 - val_loss: 0.6740 - learning_rate: 9.9815e-04
Epoch 6/200
37/37 - 1s - 22ms/step - accuracy: 0.5321 - f1_score: 0.5027 - loss: 0.6979 - val_accuracy: 0.

In [7]:
import os
import numpy as np
from sklearn.metrics import f1_score
import tensorflow as tf

# Function to compute best threshold for a model
def find_best_threshold(model, val_x, y_true_bin):
    probs = model.predict(val_x)[:, 1]
    thresholds = np.linspace(0.1, 1.0, 100)
    best_f1 = 0
    best_threshold = 0.5
    for t in thresholds:
        preds = (probs > t).astype(int)
        f1 = f1_score(y_true_bin, preds)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = t
    return best_threshold, best_f1

# Load validation data
val_npz = np.load("/kaggle/working/preprocessed/validation_MI.npz")
X_val_all, y_val = val_npz["X"], val_npz["y"]
y_val_bin = (y_val == "Right").astype(int)

# Use only EEG channels: C3, CZ, C4, PZ
eeg_indices = [1, 2, 3, 4]
X_val_raw = X_val_all[:, eeg_indices, :].transpose(0, 2, 1).astype('float32')

# Directories to check
dirs = ["/kaggle/working/models", "/kaggle/working/models2", "/kaggle/working/models3"]
# 
# Loop through models
results = {}
for model_dir in dirs:
    for file in os.listdir(model_dir):
        if file.endswith(".h5"):
            path = os.path.join(model_dir, file)
            try:
                model = tf.keras.models.load_model(path, compile=False)
                threshold, f1 = find_best_threshold(model, X_val_raw, y_val_bin)
                results[path] = (threshold, f1)
            except Exception as e:
                results[path] = f"Error: {e}"

# Print results
for model_path, (threshold, f1) in results.items():
    print(f"{model_path} → Best threshold = {threshold:.2f}, F1 score = {f1:.4f}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 300ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 4s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 378ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 255ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 386ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 603ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 401ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 273ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 301ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 256ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 4s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 386ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 264ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 393ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 640ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 413ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 277ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 305ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 394ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 6s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 385ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 259ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 392ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 621ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 409ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 279m

In [8]:
import os
import numpy as np
from sklearn.metrics import f1_score, classification_report
import tensorflow as tf

# Function to compute best threshold and generate classification report
def evaluate_model(model, val_x, y_true_str):
    # Convert true labels to binary (Right=1, Left=0)
    y_true_bin = np.array([1 if label == "Right" else 0 for label in y_true_str])
    
    # Get prediction probabilities for class 1 (Right)
    probs = model.predict(val_x, verbose=0)[:, 1]
    
    # Find best threshold
    thresholds = np.linspace(0.1, 1.0, 100)
    best_f1 = 0
    best_threshold = 0.5
    for t in thresholds:
        preds_bin = (probs > t).astype(int)
        f1 = f1_score(y_true_bin, preds_bin)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = t
    
    # Generate predictions using best threshold
    pred_labels = ["Right" if prob > best_threshold else "Left" for prob in probs]
    
    # Generate classification report
    report = classification_report(
        y_true_str, 
        pred_labels,
        target_names=["Left", "Right"],
        digits=4
    )
    
    return best_threshold, best_f1, report

# Load validation data
val_npz = np.load("/kaggle/working/preprocessed/validation_MI.npz")
X_val_all, y_val = val_npz["X"], val_npz["y"]

# Use only EEG channels: C3, CZ, C4, PZ
eeg_indices = [1, 2, 3, 4]  # Update if your channel order differs
X_val_raw = X_val_all[:, eeg_indices, :].transpose(0, 2, 1).astype('float32')

# Directories to check
dirs = [
    "/kaggle/working/models3", "/kaggle/working/models2", "/kaggle/working/models"]

# Evaluate models
results = {}
for model_dir in dirs:
    for file in os.listdir(model_dir):
        if file.endswith(".h5"):
            path = os.path.join(model_dir, file)
            try:
                model = tf.keras.models.load_model(path, compile=False)
                threshold, f1, report = evaluate_model(model, X_val_raw, y_val)
                results[path] = {
                    "threshold": threshold,
                    "f1": f1,
                    "report": report
                }
            except Exception as e:
                results[path] = f"Error: {e}"

# Print results with classification reports
for path, result in results.items():
    if isinstance(result, str):
        print(f"\n{path} → {result}")
    else:
        print(f"\n{'-'*80}")
        print(f"Model: {path}")
        print(f"Best threshold: {result['threshold']:.4f}")
        print(f"Best F1-score: {result['f1']:.4f}")
        print("\nClassification Report:")
        print(result['report'])
        print(f"{'-'*80}")

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classif


--------------------------------------------------------------------------------
Model: /kaggle/working/models3/best_final.h5
Best threshold: 0.5000
Best F1-score: 0.7273

Classification Report:
              precision    recall  f1-score   support

        Left     0.8824    0.5357    0.6667        28
       Right     0.6061    0.9091    0.7273        22

    accuracy                         0.7000        50
   macro avg     0.7442    0.7224    0.6970        50
weighted avg     0.7608    0.7000    0.6933        50

--------------------------------------------------------------------------------

--------------------------------------------------------------------------------
Model: /kaggle/working/models3/best_model1.h5
Best threshold: 0.5000
Best F1-score: 0.6471

Classification Report:
              precision    recall  f1-score   support

        Left     1.0000    0.1429    0.2500        28
       Right     0.4783    1.0000    0.6471        22

    accuracy                       

In [11]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model

# Configuration
base_path = "/kaggle/input/mtcaic3-phase-ii"
preprocessed_dir = '/kaggle/working/preprocessed'  # Directory with preprocessed .npz files
model_path = '/kaggle/working/models3/best_final.h5'
output_file = '/kaggle/working/submission.csv'
threshold = 0.5  # Prediction threshold for MI task

# Load test data and sample submission
test_df = pd.read_csv(os.path.join(base_path, 'test.csv'))
sample_sub = pd.read_csv(os.path.join(base_path, 'sample_submission.csv'))

# Fixed F1Score metric class
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name="f1_score", **kwargs):
        super().__init__(name=name, **kwargs)
        self.tp = self.add_weight(name="tp", initializer="zeros")
        self.fp = self.add_weight(name="fp", initializer="zeros")
        self.fn = self.add_weight(name="fn", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        preds = tf.argmax(y_pred, axis=1)
        labels = tf.argmax(y_true, axis=1)
        
        true_pos = tf.logical_and(tf.equal(preds, 1), tf.equal(labels, 1))
        false_pos = tf.logical_and(tf.equal(preds, 1), tf.equal(labels, 0))
        false_neg = tf.logical_and(tf.equal(preds, 0), tf.equal(labels, 1))
        
        self.tp.assign_add(tf.reduce_sum(tf.cast(true_pos, tf.float32)))
        self.fp.assign_add(tf.reduce_sum(tf.cast(false_pos, tf.float32)))
        self.fn.assign_add(tf.reduce_sum(tf.cast(false_neg, tf.float32)))

    def result(self):
        precision = self.tp / (self.tp + self.fp + tf.keras.backend.epsilon())
        recall = self.tp / (self.tp + self.fn + tf.keras.backend.epsilon())
        return 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())

    def reset_states(self):
        self.tp.assign(0.0)
        self.fp.assign(0.0)
        self.fn.assign(0.0)

# Load model with fixed metric
model = load_model(model_path, custom_objects={'F1Score': F1Score})

# Define EEG channels to extract and their indices in preprocessed data
channel_names = ['FZ','C3','CZ','C4','PZ','PO7','OZ','PO8']
target_channels = ['C3','CZ','C4','PZ']
target_indices = [channel_names.index(ch) for ch in target_channels]

# Load preprocessed test data
test_mi = np.load(os.path.join(preprocessed_dir, 'test_MI.npz'))['X']  # Shape (n_mi, 8, 2250)
test_ssvep = np.load(os.path.join(preprocessed_dir, 'test_SSVEP.npz'))['X']  # Shape (n_ssvep, 8, 1750)

# Extract target channels and transpose to (trials, time, channels)
test_mi = test_mi[:, target_indices, :].transpose(0, 2, 1)  # New shape: (n_mi, 2250, 4)
test_ssvep = test_ssvep[:, target_indices, :].transpose(0, 2, 1)  # New shape: (n_ssvep, 1750, 4)

# Get indices of MI and SSVEP trials in test_df
mi_mask = test_df['task'] == 'MI'
ssvep_mask = test_df['task'] == 'SSVEP'

# Validate counts
assert mi_mask.sum() == test_mi.shape[0], "MI trial count mismatch"
assert ssvep_mask.sum() == test_ssvep.shape[0], "SSVEP trial count mismatch"

# Generate predictions
predictions = []
# Predict MI trials
if test_mi.shape[0] > 0:
    mi_preds = model.predict(test_mi, verbose=0)
    mi_labels = ['Right' if prob[1] >= threshold else 'Left' for prob in mi_preds]
    predictions.extend(mi_labels)

# Assign dummy labels for SSVEP trials
if test_ssvep.shape[0] > 0:
    predictions.extend(['Left'] * test_ssvep.shape[0])

# Create submission file
submission = sample_sub.copy()
submission['label'] = predictions
submission.to_csv(output_file, index=False)

print(f"Submission file saved to {output_file}")
print("Prediction distribution:")
print(submission['label'].value_counts())
print(f"\nThreshold used for MI: {threshold}")

Submission file saved to /kaggle/working/submission.csv
Prediction distribution:
label
Left     133
Right     67
Name: count, dtype: int64

Threshold used for MI: 0.5


In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import f1_score, classification_report
from mne.decoding import CSP
from tensorflow import keras
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger, LearningRateScheduler
import gc

# --- 0) Config ---
BASE_PATH = '/kaggle/input/mtcaic3-phase-ii'  # Update this path
data_dir = './preprocessed'
output_dir = './models_loso'
os.makedirs(output_dir, exist_ok=True)

eeg_indices = [1, 2, 3, 4]  # C3, CZ, C4, PZ
batch_size = 64

# --- 1) Load & prepare data ---
# Load subject information from train.csv
train_df = pd.read_csv(os.path.join(BASE_PATH, 'train.csv'))
mi_train_df = train_df[train_df['task'] == 'MI'].reset_index(drop=True)
subject_ids = mi_train_df['subject_id'].values

# Load preprocessed data
train_npz = np.load(os.path.join(data_dir, 'train_MI.npz'))
X_train_all = train_npz['X']  # (2400, 8, 2250)
y_train = train_npz['y']      # (2400,)

# Filter for EEG channels
X_train_raw = X_train_all[:, eeg_indices, :].transpose(0, 2, 1).astype('float32')  # (2400, 2250, 4)

# Binarize labels
y_train_bin = (y_train == 'Right').astype(int)

# --- 2) Setup LOSO Cross-Validation ---
logo = LeaveOneGroupOut()
groups = subject_ids
n_folds = logo.get_n_splits(groups=groups)

# --- 3) Model Builders (Unchanged from original) ---
# [Include all model builder functions: build_modelA, build_modelB, build_model1, etc.]
# ... (Paste all your model builder functions here unchanged) ...

# --- 4) F1 Score Metric (Unchanged) ---
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name="f1_score", **kwargs):
        super().__init__(name=name, **kwargs)
        self.tp = self.add_weight(name="tp", initializer="zeros")
        self.fp = self.add_weight(name="fp", initializer="zeros")
        self.fn = self.add_weight(name="fn", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        preds  = tf.argmax(y_pred, axis=1)
        labels = tf.argmax(y_true, axis=1)
        self.tp.assign_add(tf.reduce_sum(tf.cast(tf.logical_and(preds == 1, labels == 1), tf.float32)))
        self.fp.assign_add(tf.reduce_sum(tf.cast(tf.logical_and(preds == 1, labels == 0), tf.float32)))
        self.fn.assign_add(tf.reduce_sum(tf.cast(tf.logical_and(preds == 0, labels == 1), tf.float32)))

    def result(self):
        p = self.tp / (self.tp + self.fp + tf.keras.backend.epsilon())
        r = self.tp / (self.tp + self.fn + tf.keras.backend.epsilon())
        return 2 * (p * r) / (p + r + tf.keras.backend.epsilon())

    def reset_states(self):
        self.tp.assign(0.0)
        self.fp.assign(0.0)
        self.fn.assign(0.0)

    # ... (Paste your F1Score class implementation here) ...

# --- 5) Cosine LR Schedule (Unchanged) ---
def cosine_lr(epoch, lr_max=5e-5, epochs=200):
    return lr_max * (1 + np.cos(np.pi * epoch / epochs)) / 2

# --- 6) Callback Factory ---
def get_callbacks(name, fold_dir):
    return [
        EarlyStopping(monitor="val_f1_score", mode="max", patience=20, restore_best_weights=True),
        ModelCheckpoint(os.path.join(fold_dir, f"best_{name}.h5")),
        CSVLogger(os.path.join(fold_dir, f"log_{name}.csv")),
        LearningRateScheduler(cosine_lr)
    ]

# --- 7) Data Augmentation Generator ---
def aug_gen(X, y, seed=0, batch_size=32):
    n = X.shape[0]
    rng = np.random.RandomState(seed)
    while True:
        idx = rng.randint(0, n, batch_size)
        bx, by = X[idx].copy(), y[idx]
        bx += rng.normal(0, 0.005, bx.shape)
        yield bx, by

# --- 8) LOSO Cross-Validation ---
# Dictionary to store results
builders = {
    'oldA': build_modelA,
    'oldB': build_modelB,
    'model1': build_model1,
    'model2': build_model2,
    'model3': build_model3,
    'model4': build_model4,
    'model5': build_model5,
    'model6': build_model6,
}

# Initialize results storage
results = {name: [] for name in builders.keys()}

for fold_idx, (train_idx, val_idx) in enumerate(logo.split(X_train_raw, y_train_bin, groups=groups)):
    subject_id = groups[val_idx[0]]
    print(f"\n{'='*50}")
    print(f"Fold {fold_idx+1}/{n_folds} - Subject: {subject_id}")
    print(f"{'='*50}")
    
    # Create output directory for fold
    fold_dir = os.path.join(output_dir, f"fold_{fold_idx+1}")
    os.makedirs(fold_dir, exist_ok=True)
    
    # Split data
    X_train_fold_raw = X_train_raw[train_idx]
    y_train_fold_bin = y_train_bin[train_idx]
    X_val_fold_raw = X_train_raw[val_idx]
    y_val_fold_bin = y_train_bin[val_idx]
    
    # --- Compute CSP using ONLY training fold data ---
    print("Computing CSP for current fold...")
    csp = CSP(n_components=4, log=False, norm_trace=False)
    
    # Prepare data for CSP: (n_trials, n_channels, n_times)
    X_train_csp_input = X_train_fold_raw.transpose(0, 2, 1).astype('float64')
    csp.fit(X_train_csp_input, y_train_fold_bin)
    W = csp.filters_[:4]  # (4, 4) filter matrix
    
    # Apply CSP to both training and validation data
    def apply_csp_fold(X):
        return np.stack([W.T @ x.T for x in X], axis=0)  # (n, n_components, n_times)
    
    X_train_fold_csp = apply_csp_fold(X_train_fold_raw).transpose(0, 2, 1).astype('float32')
    X_val_fold_csp = apply_csp_fold(X_val_fold_raw).transpose(0, 2, 1).astype('float32')
    
    # Create 2D versions for models that need it
    X_train_fold_csp_2d = X_train_fold_csp[..., np.newaxis]
    X_val_fold_csp_2d = X_val_fold_csp[..., np.newaxis]
    
    # One-hot encode labels
    y_train_fold_oh = keras.utils.to_categorical(y_train_fold_bin, 2)
    y_val_fold_oh = keras.utils.to_categorical(y_val_fold_bin, 2)
    
    # Steps per epoch
    steps_per_epoch = len(X_train_fold_raw) // batch_size
    
    # --- Train all models on current fold ---
    for name, build_fn in builders.items():
        print(f"\n>>> Training {name} on Fold {fold_idx+1}")
        
        # Clean up previous models
        tf.keras.backend.clear_session()
        gc.collect()
        
        # Select appropriate data format
        if name == 'model3':
            X_train = X_train_fold_csp_2d
            X_val = X_val_fold_csp_2d
            input_shape = X_train.shape[1:]
            train_gen = aug_gen(X_train, y_train_fold_oh, seed=fold_idx, batch_size=batch_size)
        elif name in ['model4', 'model5']:
            X_train = X_train_fold_csp
            X_val = X_val_fold_csp
            input_shape = X_train.shape[1:]
            train_gen = aug_gen(X_train, y_train_fold_oh, seed=fold_idx, batch_size=batch_size)
        else:
            X_train = X_train_fold_raw
            X_val = X_val_fold_raw
            input_shape = X_train.shape[1:]
            train_gen = aug_gen(X_train, y_train_fold_oh, seed=fold_idx, batch_size=batch_size)
        
        # Build model
        model = build_fn(input_shape)
        
        # Train model
        history = model.fit(
            train_gen,
            steps_per_epoch=steps_per_epoch,
            epochs=200,
            validation_data=(X_val, y_val_fold_oh),
            callbacks=get_callbacks(name, fold_dir),
            verbose=2
        )
        
        # Evaluate
        preds = np.argmax(model.predict(X_val), axis=1)
        f1 = f1_score(y_val_fold_bin, preds)
        print(f"{name} → Fold {fold_idx+1} Val F1 = {f1:.4f}")
        results[name].append(f1)
        
        # Save fold results
        np.save(os.path.join(fold_dir, f"results_{name}.npy"), np.array([f1]))

# --- 9) Final Results ---
print("\n\n=== Final LOSO Results ===")
for name, f1_scores in results.items():
    mean_f1 = np.mean(f1_scores)
    std_f1 = np.std(f1_scores)
    print(f"{name}:")
    print(f"  F1 Scores: {', '.join([f'{s:.4f}' for s in f1_scores])}")
    print(f"  Mean F1 = {mean_f1:.4f} ± {std_f1:.4f}\n")

# Save all results
np.save(os.path.join(output_dir, "all_results.npy"), results)
print("LOSO complete!")


Fold 1/30 - Subject: S1
Computing CSP for current fold...
Computing rank from data with rank=None
    Using tolerance 3.2e+03 (2.2e-16 eps * 4 dim * 3.6e+18  max singular value)
    Estimated rank (data): 4
    data: rank 4 computed from 4 data channels with 0 projectors
Reducing data rank from 4 -> 4
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.

>>> Training oldA on Fold 1
Epoch 1/200
36/36 - 8s - 235ms/step - accuracy: 0.5087 - f1_score: 0.5212 - loss: 0.8719 - val_accuracy: 0.5500 - val_f1_score: 0.2500 - val_loss: 0.6974 - learning_rate: 0.0010
Epoch 2/200
36/36 - 1s - 29ms/step - accuracy: 0.5056 - f1_score: 0.5105 - loss: 0.7853 - val_accuracy: 0.4625 - val_f1_score: 0.3944 - val_loss: 0.7013 - learning_rate: 9.9994e-04
Epoch 3/200
36/36 - 1s - 29ms/step - accuracy: 0.4918 - f1_score: 0.4924 - loss: 0.7361 - val_accuracy: 0.5500 - val_f1_score: 0.4194 - val_loss: 0.6966 - learning_rate: 9.9969e-04
Epoch 4/200
36/36 - 1s 